# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

In [1]:
import pandas as pd
import numpy as np
import random

# 2. ML-mетрики качества

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [2]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)[:k]
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [3]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

print(f'hit_rate: {hit_rate(recommended_list, bought_list)}')
for k in range(1, 6):
    print(f'hit_rate_at_k (k={k}): {hit_rate_at_k(recommended_list, bought_list, k)}')

hit_rate: 1
hit_rate_at_k (k=1): 0
hit_rate_at_k (k=2): 0
hit_rate_at_k (k=3): 1
hit_rate_at_k (k=4): 1
hit_rate_at_k (k=5): 1


### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [4]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)[:k]
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended_ = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended_) / prices_recommended_.sum()
    return precision

In [5]:
random.seed(100)

recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
prices_recommended = [random.randint(50, 500) for i in range(1, len(recommended_list)+1)]

print(f'precision: {precision(recommended_list, bought_list)}\n')

for k in range(1, 6):
    print(f'k={k}')
    print(f'precision_at_k: {precision_at_k(recommended_list, bought_list, k)}')
    print(f'money_precision_at_k: {money_precision_at_k(recommended_list, bought_list, prices_recommended, k)}\n')

precision: 0.2

k=1
precision_at_k: 0.0
money_precision_at_k: 1.0

k=2
precision_at_k: 0.0
money_precision_at_k: 0.30317848410757947

k=3
precision_at_k: 0.3333333333333333
money_precision_at_k: 0.17945007235890015

k=4
precision_at_k: 0.5
money_precision_at_k: 0.5004405286343613

k=5
precision_at_k: 0.4
money_precision_at_k: 0.44583987441130296



### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [6]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)[:k]
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended_ = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    recall = np.dot(flags, prices_recommended_) / prices_bought.sum()
    return recall

In [7]:
random.seed(100)

recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
prices_recommended = [random.randint(50, 500) for i in range(1, len(recommended_list)+1)]
prices_bought = np.isin(recommended_list, bought_list) * prices_recommended

print(f'recall: {recall(recommended_list, bought_list)}\n')

for k in range(1, 6):
    print(f'k={k}')
    print(f'recall_at_k: {recall_at_k(recommended_list, bought_list, k)}')
    print(f'money_recall_at_k: {money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k)}\n')

recall: 0.5

k=1
recall_at_k: 0.0
money_recall_at_k: 0.21830985915492956

k=2
recall_at_k: 0.0
money_recall_at_k: 0.21830985915492956

k=3
recall_at_k: 0.3333333333333333
money_recall_at_k: 0.21830985915492956

k=4
recall_at_k: 0.5
money_recall_at_k: 1.0

k=5
recall_at_k: 0.5
money_recall_at_k: 1.0



---

# Метрики ранжирования

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантных среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [8]:
recommended_list = [221, 21, 3234, 1, 234, 234, 234, 666] #id товаров
bought_list = [1, 221, 3, 4, 5, 6, 7, 8, 9]

In [9]:
def ap_k(recommended_list, bought_list, k=5):
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [10]:
ap_k(recommended_list, bought_list, k=5)

0.25

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [11]:
# теперь список из 3 пользователей
recommended_list_3_users = [
    [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345], 
    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11], 
]

bought_list_3_users = [
    [521, 32, 143],  # юзер 1
    [143, 156, 991, 43, 11], # юзер 2
    [991, 1, 2], # юзер 3
]

In [12]:
def map_k(recommended_list, bought_list, k=5):
    
    result = 0
    for itm in zip(recommended_list, bought_list):
        result += ap_k(itm[0], itm[1], k=5)
    
    return result / len(recommended_list)

In [13]:
map_k(recommended_list_3_users, bought_list_3_users, k)

0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [14]:
def dcg_at_k(recommended_list, bought_list, k=5, is_ideal=False):
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)

    if is_ideal == False:
        is_bought = np.isin(recommended_list, bought_list) * 1
    else:
        is_bought = np.full(len(recommended_list), 1)
    
    sum_ = sum([value / idx if idx in [1, 2] else value / np.log2(idx) for idx, value in enumerate(is_bought, 1)])
    return sum_/len(recommended_list)

def ndcg_at_k(recommended_list, bought_list, k=5):
    return dcg_at_k(recommended_list, bought_list, k=5, is_ideal=False) / dcg_at_k(recommended_list, bought_list, k=5, is_ideal=True)

In [15]:
recommended_list = [221, 21, 3234, 1, 234, 234, 234, 666] #id товаров
bought_list = [1, 221, 3, 4, 5, 6, 7, 8, 9]

print(f'dcg_at_k: {dcg_at_k(recommended_list, bought_list, k=5)}')
print(f'ndcg_at_k: {ndcg_at_k(recommended_list, bought_list, k=5)}')

dcg_at_k: 0.3
ndcg_at_k: 0.489938890671454


### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [16]:
def reciprocal_rank(recommended_list, bought_list, k=1):

    mrr = []
    for itm in zip(recommended_list, bought_list):
        flags = np.isin(np.array(itm[0])[:k], np.array(itm[1]))
        relevant_indexes = np.nonzero(flags)[0]
        
        if len(relevant_indexes) == 0:
            mrr.append(0)
        else:
            mrr.append(1 / (relevant_indexes[0] + 1))
    return np.mean(mrr)

In [17]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.6666666666666666

# 3. Связь бизнес-метрик, ML-метрик и функции потерь(loss)

- **loss** - то, что оптимизирует модель (RMSE - Root Mean Squared Error)
- **ML-метрика** - то, как мы измеряем качество на test (money precision@5)
- **Бизнес-метрика** - то, что хочет оптимизировать бизнес (выручка)

В идеале loss = ML-метрика = бизнес-метрика, но это возмонжо в очень редких случаях

Надеемся, что:  
    *Оптимизация loss --> рост ML-метрик --> рост бизнес-метрик*

### Пример: рекомендательная система товаров на сайте  
      
На сайте есть баннер, на котором мы можем разместить 5 товаров. Надо порекомендовать эти 5 товаров персонально каждому юзеру

*Шаг 1: Определим бизнес-метрику*  
Бизнес хочет максимизировать выручку --> бизнес-метрика - **Выручка**  

*Шаг 2: Разложим ее на составляющие*  
Выручка =   
       Средний чек * кол-во покупок =   
       Средний чек * (число юзеров * конверсия из захода на сайт в заказ) =
       Число юзеров * (Средний чек * конверсия из захода на сайт в заказ)
       
Рекомендательная система влияет только на (Средний чек * конверсия из захода на сайт в заказ).   
Хорошим приближением этого является **money precision@5** - ML-метрика

*Шаг 3: loss*  
Это существенно сложнее. Персонализированные ML-модели не умеют напрямую оптимизировать конверсию. Стандартный loss - **RMSE**. Можно для начала попробовать его. 

P.S. Если вы хотите приблизить RMSE к money precision@5, то можно посчитать weighted RMSE, где вес каждого наблюдения = стоимость товара.   
P.S.S. Про другие виды loss будет рассказано в курсе

# Полезные ссылки

Статьи: https://docs.google.com/document/d/16L1u5zuQyT5rSAQNk2Q7Zk5KzqlPqWTpYa2cPK3QU0Y/edit?usp=sharing

# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**

### 1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

Kendall’s tau: https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient

Spearman’s rho: https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient